In [10]:
import os
import glob
import pandas as pd
from config import *

parquet_files = glob.glob(os.path.join(ROOT_DIR/'data', '*.parquet'))

df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)

# Concatenate all the DataFrames into a single DataFrame
df = pd.concat(df_list, ignore_index=True)


ModuleNotFoundError: No module named 'config'

In [1]:
from datasets import load_dataset

dataset = load_dataset("monash_tsf", "traffic_hourly")

/Users/lando/Projects/auctioneer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 862/862 [00:00<00:00, 6652.16 examples/s]


In [2]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [3]:
from functools import lru_cache

import pandas as pd
import numpy as np


@lru_cache(10_000)
def convert_to_pandas_period(date, freq):
    return pd.Period(date, freq)

def transform_start_field(batch, freq):
    batch["start"] = [convert_to_pandas_period(date, freq) for date in batch["start"]]
    return batch

In [4]:
freq = "1H"
prediction_length = 48

train_example = dataset["train"][0]
validation_example = dataset["validation"][0]

assert len(train_example["target"]) + prediction_length == len(
    dataset["validation"][0]["target"]
)

In [5]:
from functools import partial

train_dataset.set_transform(partial(transform_start_field, freq=freq))
test_dataset.set_transform(partial(transform_start_field, freq=freq))

In [7]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

num_of_variates = len(train_dataset)

train_grouper = MultivariateGrouper(max_target_dim=num_of_variates)
test_grouper = MultivariateGrouper(
    max_target_dim=num_of_variates,
    num_test_dates=len(test_dataset) // num_of_variates, # number of rolling test windows
)

multi_variate_train_dataset = train_grouper(train_dataset)
multi_variate_test_dataset = test_grouper(test_dataset)

In [9]:
multi_variate_train_example = multi_variate_train_dataset[0]
print("multi_variate_train_example['target'].shape =", multi_variate_train_example["target"].shape)

multi_variate_train_example['target'].shape = (862, 17448)
